## Computational Assignment 3
### Jacob Holmes

### Install Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import networkx as nx
import pyomo.environ as pyo

### Importing data as dataframes

In [2]:
# Modified data import to only have numbers for the Machine index and Job index
pt_df = pd.read_csv('data/ca3-p2.csv', index_col=0)

#print(pt_df)

customers_df = pd.read_csv('data/ca3-p3-customers.csv', index_col=0)

#print(customers_df)

facilities_df = pd.read_csv('data/ca3-p3-facilities.csv', index_col=0)

#print(facilities_df)

### Including problem parameters

In [3]:
# Problem 2

m_num = 4 # number of machines
n = 6 # number of jobs


# Problem 7 

b = 5 # number of facilities to locate
a = 1.5 # maximum Euclidean distance for coverage from a facility to a customer


## Problem 2

In [4]:
# Creating needed sets and other parameters

#  Problem 2
Jobs = list(map(int, pt_df.columns))
#print(Jobs)

Machines = list(pt_df.index)
#print(Machines)

processing_time = {(i,int(j)): float(pt_df.loc[i,j]) for i in Machines for j in pt_df.columns}
#print(processing_time)

big_m = sum(processing_time[i,j] for i in Machines for j in Jobs)
#print(big_m)

In [5]:
# Defining the model for Problem 2

model = pyo.ConcreteModel()

model.MACHINES = pyo.Set(initialize=Machines)
model.JOBS = pyo.Set(initialize=Jobs)

model.p = pyo.Param(model.MACHINES, model.JOBS, initialize=processing_time)

model.x = pyo.Var(model.MACHINES, model.JOBS, within = pyo.NonNegativeIntegers)
model.y = pyo.Var(model.MACHINES, model.JOBS, model.JOBS, within = pyo.Binary)
model.w = pyo.Var(within = pyo.NonNegativeReals)

In [6]:
# Adding Objective Function
def obj_rule(model):
    return model.w

model.OBJ = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

In [7]:
# Adding Precedence Constraints for machine order for each job
def precedence_rule(m, i, j):
    if i == m_num:
        return m.x[i,j] + m.p[i,j] <= m.w       
    else:
        return m.x[i,j] + m.p[i,j] <= m.x[i+1,j]
    

model.PRECEDENCE_CONST = pyo.Constraint(model.MACHINES, model.JOBS, rule=precedence_rule)

#model.PRECEDENCE_CONST.pprint()

In [8]:
# Adding one job per machine at a time constraint and Order Constraints based on this

def machine_assignment_rule(m, i, j, k):
    if j == k:
        return pyo.Constraint.Skip
    else:
        return m.x[i,j] + m.p[i,j] <= m.x[i,k] + big_m * (1 - m.y[i,j,k])
    
def order_rule(m, i, j, k):
    if j == k:
        return pyo.Constraint.Skip
    else:
        return m.y[i,j,k] + m.y[i,k,j] == 1
    
model.MACHINE_ASSIGNMENT_CONST = pyo.Constraint(model.MACHINES, model.JOBS, model.JOBS, rule=machine_assignment_rule)
model.ORDER_CONST = pyo.Constraint(model.MACHINES, model.JOBS, model.JOBS, rule=order_rule)


In [9]:
# Solving the model

opt = pyo.SolverFactory('cplex')
results = opt.solve(model)

print("The optimal makespan is: ", pyo.value(model.OBJ))
for i in model.MACHINES:
    for j in model.JOBS:
        print(f"Start time of job {j} on machine {i} (x[{i},{j}]): ", pyo.value(model.x[i,j]))


The optimal makespan is:  114.0
Start time of job 1 on machine 1 (x[1,1]):  77.0
Start time of job 2 on machine 1 (x[1,2]):  21.0
Start time of job 3 on machine 1 (x[1,3]):  29.0
Start time of job 4 on machine 1 (x[1,4]):  0.0
Start time of job 5 on machine 1 (x[1,5]):  16.0
Start time of job 6 on machine 1 (x[1,6]):  40.0
Start time of job 1 on machine 2 (x[2,1]):  99.0
Start time of job 2 on machine 2 (x[2,2]):  29.0
Start time of job 3 on machine 2 (x[2,3]):  40.0
Start time of job 4 on machine 2 (x[2,4]):  16.0
Start time of job 5 on machine 2 (x[2,5]):  21.0
Start time of job 6 on machine 2 (x[2,6]):  70.0
Start time of job 1 on machine 3 (x[3,1]):  109.0
Start time of job 2 on machine 3 (x[3,2]):  44.0
Start time of job 3 on machine 3 (x[3,3]):  73.0
Start time of job 4 on machine 3 (x[3,4]):  19.0
Start time of job 5 on machine 3 (x[3,5]):  29.0
Start time of job 6 on machine 3 (x[3,6]):  80.0
Start time of job 1 on machine 4 (x[4,1]):  112.0
Start time of job 2 on machine 4 (x[

## Problem 7

In [10]:
# Creating a distance function for Problem 7 to be used inside constraints
def distance(fac, cust):
    return np.sqrt((facilities_df.loc[fac,'x'] - customers_df.loc[cust,'x'])**2 + (facilities_df.loc[fac,'y'] - customers_df.loc[cust,'y'])**2)

In [11]:
#Creating needed sets for Problem 7

Facilities = list(facilities_df.index)
Customers = list(customers_df.index)


#print(Facilities)
#print(Customers)

In [12]:
# Defining the model for Problem 7

model_7 = pyo.ConcreteModel()

model_7.CUSTOMERS = pyo.Set(initialize=Customers)
model_7.FACILITIES = pyo.Set(initialize=Facilities)

In [13]:
# Defining decision variables

model_7.x = pyo.Var(model_7.FACILITIES, within = pyo.Binary)
model_7.y = pyo.Var(model_7.CUSTOMERS, within = pyo.Binary)

In [14]:
# Adding Objective Function

def obj_rule(m):
    return sum(m.y[c] for c in m.CUSTOMERS)

model_7.OBJ = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

In [15]:
# Adding Coverage Constraints

def coverage_rule(m, f, c):
    return (1-m.x[f]) <= m.y[c] if distance(f,c) <= a else pyo.Constraint.Skip


model_7.COVERAGE_CONST = pyo.Constraint(model_7.FACILITIES, model_7.CUSTOMERS, rule=coverage_rule)

#model_7.COVERAGE_CONST.pprint()

In [16]:
# Adding Removal Constraint for the number of facilities allowed to remove

def removal_rule(m):
    return sum(m.x[f] for f in m.FACILITIES) == b

model_7.REMOVAL_CONST = pyo.Constraint(rule=removal_rule)

In [18]:
# Solving the model

results_7 = opt.solve(model_7)

print("The optimal number of covered customers is: ", pyo.value(model_7.OBJ))

print()

print("The customers covered are: ")
for c in model_7.CUSTOMERS:
    if pyo.value(model_7.y[c]) > 0.5:
        print(c)

print()

print("The facilities removed are: ")
for f in model_7.FACILITIES:
    if pyo.value(model_7.x[f]) > 0.5:
        print(f)

The optimal number of covered customers is:  63.0

The customers covered are: 
1
2
3
4
5
7
8
12
13
14
15
16
17
18
19
23
24
25
29
30
34
35
36
45
46
50
51
61
62
63
71
72
73
74
82
83
84
89
90
92
93
94
95
96
97
100
101
102
103
104
105
106
107
108
109
111
112
114
115
117
118
119
120

The facilities removed are: 
2
3
4
10
13
